In [1]:
import os
import torch
import wandb
from wandb.integration.ultralytics import add_wandb_callback
from ultralytics import YOLO
from IPython.display import display, Image

wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.empty_cache()
device

'cuda'

In [3]:
wandb.login()

wandb: Currently logged in as: mukhlisraza (mrlabs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
# Sweep configuration
sweep_configuration = {
    "name": "smart-office-final-sweep",
    "method": "bayes",
    "metric": {
        "name": "metrics/mAP50-95(B)",
        "goal": "maximize"
    },
    "parameters": {
        "model": {
            "values": ['yolov8n', 'yolov8s', 'yolov8m', 'yolov8l', 'yolov8x']
        },
        "pretrained": {
            "values": [True, False]
        },
        "lr0": {
            "min": 0.0001, 
            "max": 0.01
        },
        "batch_size": {
            "values": [4, 8, 12, 16]
        },
        "optimizer": {
            "values": ["AdamW", "SGD", "Adam", "RMSProp"]
        },
        "imgsz": {
            "values": [640]
        },
        "epochs": {
            "values": [50, 100, 150, 200]
        },
        "val": {
            "values": [True]
        },
        "plots": {
            "values": [True]
        },
        "cos_lr": {
            "values": [True, False]
        },
        "amp": {
            "values": [True, False]
        },
        "single_cls": {
            "values": [False]
        },

    }
}

In [5]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project="smart-office")

Create sweep with ID: rakeqzh8
Sweep URL: https://wandb.ai/mrlabs/smart-office/sweeps/rakeqzh8


In [6]:
sweep_id

'rakeqzh8'

In [ ]:
def train():
    with wandb.init() as run:
        config = run.config
        model = YOLO(f"{config.model}.pt").to('cuda' if torch.cuda.is_available() else 'cpu')
            
        try:
            # Start training with the given configuration
            results = model.train(
                project="office-objects-detections",
                data=f'../data/data.yaml',
                epochs=config.epochs,
                imgsz=config.imgsz,
                batch=config.batch_size,
                lr0=config.lr0,
                workers=8,
                plots=config.plots,
                pretrained=config.pretrained,
                device='cuda' if torch.cuda.is_available() else 'cpu',
                optimizer=config.optimizer,
                name='yolos-updated-dataset-trail2'
            )
        
            # Log the results
            wandb.log({"loss": results})
        
        except Exception as e:
            print(f"An error occurred: {e}")

# Use the existing sweep ID to start the sweep
wandb.agent(sweep_id, function=train, count=5)

wandb: Agent Starting Run: ngyncr2z with config:
wandb: 	amp: True
wandb: 	batch_size: 16
wandb: 	cos_lr: False
wandb: 	epochs: 100
wandb: 	imgsz: 640
wandb: 	lr0: 0.009246473782385474
wandb: 	model: yolov8x
wandb: 	optimizer: RMSProp
wandb: 	plots: True
wandb: 	pretrained: False
wandb: 	single_cls: False
wandb: 	val: True
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Ultralytics 8.3.173  Python-3.11.7 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.009246473782385474, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolos-updated-dataset-trail2, nbs=64, nms=False, opset=None, optimize=False, optimizer=RMSProp, overlap_mask=True, patience=100, persp

train: Scanning D:\CODES\office-object-detection-hackathon_challenge\data\train\labels.cache... 444 images, 0 backgroun


val: Fast image access  (ping: 0.10.0 ms, read: 480.3115.7 MB/s, size: 68.3 KB)


val: Scanning D:\CODES\office-object-detection-hackathon_challenge\data\valid\labels.cache... 42 images, 0 backgrounds,


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     56/100      12.4G        nan        nan        nan        433        640: 100%|██████████| 28/28 [00:11<00:00,  2.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<0

                   all         42       1300          0          0          0          0



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     57/100      12.4G        nan        nan        nan        542        640:  50%|█████     | 14/28 [00:05<00:05,  2.